### Quora Data Challenge

In [162]:
import io
import numpy as np
import pandas as pd
import pandasql as ps
import plotly.tools as tls
import plotly.offline as py
import pandas_profiling as pp
import plotly.graph_objs as go
import matplotlib.pyplot as plt
import plotly.figure_factory as ff
py.init_notebook_mode(connected=True)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

Suppose you are a Data Scientist on the Mobile team at Quora.<br>The team has just introduced a new UI design to the Quora app.<br>The goal of the new design is to increase user engagement (measured by minutes spent on site). <br>The team ran an A/B test to evaluate the change.<br> Using the data, help the team understand the impact of the UI change better.

Tables provided are as follows:
    1. t1_user_active_min.csv
    2. t2_user_variant.csv
    3. t3_user_active_min_pre.csv
    4. t4_user_attributes.csv

In [163]:
# #function  for pie plot for customer attrition types
# def plot_pie(column) :
    
#     '''
#     This function plots the different pie graphs for categorical data
#     '''
    
#     trace1 = go.Pie(values  = churn[column].value_counts().values.tolist(),
#                     labels  = churn[column].value_counts().keys().tolist(),
#                     hoverinfo = "label+percent+name",
#                     domain  = dict(x = [0,.48]),
#                     name    = "Churn Customers",
#                     marker  = dict(line = dict(width = 2,
#                                                color = "rgb(243,243,243)")
#                                   ),
#                     hole    = .6
#                    )
#     trace2 = go.Pie(values  = not_churn[column].value_counts().values.tolist(),
#                     labels  = not_churn[column].value_counts().keys().tolist(),
#                     hoverinfo = "label+percent+name",
#                     marker  = dict(line = dict(width = 2,
#                                                color = "rgb(243,243,243)")
#                                   ),
#                     domain  = dict(x = [.52,1]),
#                     hole    = .6,
#                     name    = "Non churn customers" 
#                    )


#     layout = go.Layout(dict(title = column + " distribution in customer attrition ",
#                             plot_bgcolor  = "rgb(243,243,243)",
#                             paper_bgcolor = "rgb(243,243,243)",
#                             annotations = [dict(text = "churn customers",
#                                                 font = dict(size = 13),
#                                                 showarrow = False,
#                                                 x = .15, y = .5),
#                                            dict(text = "Non churn customers",
#                                                 font = dict(size = 13),
#                                                 showarrow = False,
#                                                 x = .88,y = .5
#                                                )
#                                           ]
#                            )
#                       )
#     data1 = [trace1,trace2]
#     fig  = go.Figure(data = data1,layout = layout)
#     py.iplot(fig)

# #for all categorical columns plot pie
# for i in cat_cols :
#     plot_pie(i)

In [164]:
# reading the data sets
t1_user_active_min     = pd.read_csv("t1_user_active_min.csv")
t2_user_variant        = pd.read_csv("t2_user_variant.csv")
t3_user_active_min_pre = pd.read_csv("t3_user_active_min_pre.csv")
t4_user_attributes     = pd.read_csv("t4_user_attributes.csv")
df_list = [t1_user_active_min,t2_user_variant,t3_user_active_min_pre,t4_user_attributes]

In [165]:
t1_user_active_min.head()

,uid,dt,active_mins
0,0,2019-02-22,5.0
1,0,2019-03-11,5.0
2,0,2019-03-18,3.0
3,0,2019-03-22,4.0
4,0,2019-04-03,9.0


In [166]:
t2_user_variant.head()

,uid,variant_number,dt,signup_date
0,0,0,2019-02-06,2018-09-24
1,1,0,2019-02-06,2016-11-07
2,2,0,2019-02-06,2018-09-17
3,3,0,2019-02-06,2018-03-04
4,4,0,2019-02-06,2017-03-09


In [167]:
t2_user_variant['variant_number'].value_counts()

0    40000
1    10000
Name: variant_number, dtype: int64

People in control group: 80 percent <br>
People in treatment group: 20 percent

In [168]:
user_control = []
user_treatment = []
for index,row in t2_user_variant.iterrows():
    if row['variant_number'] == 0:
        user_control.append(row['uid'])
    else:
        user_treatment.append(row['uid'])

In [169]:
t3_user_active_min_pre.head()

,uid,dt,active_mins
0,0,2018-09-24,3.0
1,0,2018-11-08,4.0
2,0,2018-11-24,3.0
3,0,2018-11-28,6.0
4,0,2018-12-02,6.0


In [170]:
t4_user_attributes.head()

,uid,gender,user_type
0,0,male,non_reader
1,1,male,reader
2,2,male,non_reader
3,3,male,non_reader
4,4,male,non_reader


### Hypothesis:
1. The AB Testing is intended to increase the user engagement in terms of time.
2. Randomized people are used to divide into control and treatment groups.

### KPI for AB Test to look for between treatment and control group:
1. Active time        - increased/ decreased
2. Frequency of visit - increased/decreased
3. Segregation of above two points for old/new users, male/female, reader/non-readers.
4. I will try to look for the combination of all the other factors

#### Creating Analytical Data Table for the user variant - combination of gender/reader/non-reader/days_since_signup

In [171]:
var_gen_ageinquora = pd.merge(t2_user_variant,t4_user_attributes,how = "inner", on = "uid")

In [172]:
var_gen_ageinquora.head()

,uid,variant_number,dt,signup_date,gender,user_type
0,0,0,2019-02-06,2018-09-24,male,non_reader
1,1,0,2019-02-06,2016-11-07,male,reader
2,2,0,2019-02-06,2018-09-17,male,non_reader
3,3,0,2019-02-06,2018-03-04,male,non_reader
4,4,0,2019-02-06,2017-03-09,male,non_reader


In [173]:
var_gen_ageinquora_control = var_gen_ageinquora[var_gen_ageinquora['variant_number'] == 0]
var_gen_ageinquora_treatment = var_gen_ageinquora[var_gen_ageinquora['variant_number'] == 1]
cat_cols = ['gender','user_type']
cont_cols = ['days_since_joined_till_AB_tested']

#### Checking the ratio in treatment and control group

In [174]:
treat_gender = pd.DataFrame(var_gen_ageinquora_treatment['gender'].value_counts())
treat_gender.rename(columns = {'gender':'count_treat_gender'}, inplace = True)
treat_gender.reset_index(inplace = True)
treat_gender.rename(columns = {'index':'gender'}, inplace = True)

In [175]:
control_gender = pd.DataFrame(var_gen_ageinquora_control['gender'].value_counts())
control_gender.rename(columns = {'gender':'count_control_gender'}, inplace = True)
control_gender.reset_index(inplace = True)
control_gender.rename(columns = {'index':'gender'}, inplace = True)

In [176]:
total_gender = pd.DataFrame(var_gen_ageinquora['gender'].value_counts())
total_gender.rename(columns = {'gender':'count_control_gender'}, inplace = True)
total_gender.reset_index(inplace = True)
total_gender.rename(columns = {'index':'gender'}, inplace = True)

In [177]:
treat_user_type = pd.DataFrame(var_gen_ageinquora_treatment['user_type'].value_counts())
treat_user_type.rename(columns = {'user_type':'count_treat_user_type'}, inplace = True)
treat_user_type.reset_index(inplace = True)
treat_user_type.rename(columns = {'index':'user_type'}, inplace = True)

In [178]:
control_user_type = pd.DataFrame(var_gen_ageinquora_control['user_type'].value_counts())
control_user_type.rename(columns = {'user_type':'count_control_user_type'}, inplace = True)
control_user_type.reset_index(inplace = True)
control_user_type.rename(columns = {'index':'user_type'}, inplace = True)

In [179]:
total_user_type = pd.DataFrame(var_gen_ageinquora['user_type'].value_counts())
total_user_type.rename(columns = {'user_type':'count_total_user_type'}, inplace = True)
total_user_type.reset_index(inplace = True)
total_user_type.rename(columns = {'index':'user_type'}, inplace = True)

In [180]:
gender_comparision = pd.merge(treat_gender, control_gender, on = "gender",how = "inner")
user_type_comparision = pd.merge(treat_user_type, control_user_type, on = "user_type",how = "inner")

In [181]:
gender_comparision['treat_ratio'] = gender_comparision['count_treat_gender'] / gender_comparision['count_treat_gender'].sum()
gender_comparision['control_ratio'] = gender_comparision['count_control_gender'] / gender_comparision['count_control_gender'].sum()

In [182]:
gender_comparision.drop(columns = ['count_treat_gender','count_control_gender'])

,gender,treat_ratio,control_ratio
0,male,0.552,0.555925
1,female,0.287,0.290175
2,unknown,0.161,0.153900


In [183]:
user_type_comparision['treat_ratio'] = user_type_comparision['count_treat_user_type'] / user_type_comparision['count_treat_user_type'].sum()
user_type_comparision['control_ratio'] = user_type_comparision['count_control_user_type'] / user_type_comparision['count_control_user_type'].sum()

In [184]:
user_type_comparision.drop(columns = ['count_treat_user_type','count_control_user_type'])

,user_type,treat_ratio,control_ratio
0,non_reader,0.7367,0.717475
1,reader,0.1269,0.168325
2,new_user,0.1235,0.091325
3,contributor,0.0129,0.022875
